## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [ ]:
import requests

In [ ]:
def get_the_highest():
    '''
    Нахожу самую дорогую валюту в данных, полученных от https://www.cbr-xml-daily.ru/daily_json.js.
    Возвращаю ее название.
    Входные параметры не нужны.
    '''
    currencies = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
    value = 0.0
    for currency in currencies['Valute'].values():
        # если нужно учесть номинал, то вместо currency['Value'] указываем currency['Value']/currency['Nominal']
        if currency['Value'] > value:
            value = currency['Value']
            key = currency['Name']
    return key

In [ ]:
get_the_highest()

## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [ ]:
class Rate:
    def __init__(self, format_='value', diff=False):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        – информация о текущем курсе при self.format = 'value' и self.diff = False
        Rate('value').make_format('EUR')
        79.4966

        – информация об изменении курса за последние сутки при self.format = 'value' и self.diff = True
        Rate('value', True).make_format('EUR')
        -0.0969
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return round(response[currency]['Value'] - response[currency]['Previous'],4) if self.diff else response[currency]['Value'] 
        
        return 'Error'
    
    def eur(self):
        """Возвращает:
        — полную информацию о евро при self.format = 'full'
        — курс евро на сегодня при self.format = 'value' и self.diff = False
        — информацию об изменении курса евро за сутки при self.format = 'value' и self.diff = True
        """
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает:
        — полную информацию о долларе США при self.format = 'full'
        — курс доллара США на сегодня при self.format = 'value' и self.diff = False
        — информацию об изменении курса доллара США за сутки при self.format = 'value' и self.diff = True
        """
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает:
        — полную информацию о бразильском реале при self.format = 'full'
        — курс бразильского реала на сегодня при self.format = 'value' и self.diff = False
        — информацию об изменении курса бразильского реала за сутки при self.format = 'value' и self.diff = True
        """
        return self.make_format('BRL')

In [ ]:
print(Rate('value').make_format('EUR'))
print(Rate('value', True).make_format('EUR'))

## Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.


In [ ]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [ ]:
class Designer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self, international_award=False):
        # для каждой аккредитации увеличиваем счетчик на 1
        # при получении международной премии увеличиваем счетчик на 2
        self.seniority += 2 if international_award else 1
    
        # условие повышения сотрудника из презентации
        if self.seniority >= 7:
            self.grade_up()
            self.seniority %= 7
        
        # публикация результатов
        return self.publish_grade()

In [ ]:
d = Designer('Юра', 0)
for i in range(5):
    d.check_if_it_is_time_for_upgrade()
    print(f'Обычная аккредитация: {d.seniority}')
    d.check_if_it_is_time_for_upgrade(international_award=True)
    print(f'Международная премия: {d.seniority}')